### Embedding layer

Keras предлагает слой Embedding, который можно использовать в
моделях нейронных сетей для обработки текстовых данных. Он требует, чтобы
входные данные были закодированы целыми числами, так что каждое слово
представлено уникальным целым числом. Эта стадия подготовки данных может
быть выполнена с использованием API Tokenizer, также предоставляемого
Keras.
Слой Embedding инициализируется случайными весами и производит
векторизацию для всех слов в наборе учебных данных.
Это гибкий слой, который можно использовать различными способами,
такими как:
1. Его можно использовать отдельно, чтобы изучить векторизацию слов,
которое может быть сохранено и использовано в другой модели позже.
2. Он может использоваться как часть модели глубокого обучения, в
которой векторизацию изучается вместе с самой моделью.
3. Его можно использовать для загрузки предварительно подготовленной
модели векторизации слов, типа передачи обучения.

***Слой векторизации Embedding определяется как первый скрытый уровень
сети***

In [13]:
import os
import sys
#pip loguru
from loguru import logger
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'  # Suppress TensorFlow logging (1)

In [14]:
logger.remove()  # Remove the default logger
logger.add(sys.stdout, level="INFO")  # Add a new logger that outputs to stderr

1

In [15]:
from numpy import array
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential 
from tensorflow.keras.layers import Dense, Flatten, Embedding

Create two sets:
* Features - the corpus of words
* Labels - in case of positive will be 1 in case of negative 0.

In [21]:
corpus = ['Well done!', 'Good work', 'Great effort', 'nice work', 'Excellent!',
'Weak', 'Poor effort!', 'not good', 'poor work', 'Could have done better.']
labels = array([1,1,1,1,1,0,0,0,0,0])

Create one-hot encoding on each word in corpus. The vocabulary size will be grather in twise for reduce propobility of ovelap values from hash function

In [30]:
vocab_size = 50
encoded_docs = [one_hot(d, vocab_size) for d in corpus]
logger.info(encoded_docs)

2025-04-04 13:16:26.057 | INFO     | __main__:<module>:3 - [[25, 26], [49, 18], [48, 4], [39, 18], [27], [6], [42, 4], [11, 49], [42, 18], [31, 24, 26, 39]]


The output vectors is not in same dimention so, let`s to extend all smaller vectros by padding zero`s to the end of each vector

In [31]:
max_length = 4
padded_docs=pad_sequences(encoded_docs,maxlen=max_length, padding='post')
logger.info(padded_docs)

2025-04-04 13:16:28.420 | INFO     | __main__:<module>:3 - [[25 26  0  0]
 [49 18  0  0]
 [48  4  0  0]
 [39 18  0  0]
 [27  0  0  0]
 [ 6  0  0  0]
 [42  4  0  0]
 [11 49  0  0]
 [42 18  0  0]
 [31 24 26 39]]


Create model:
* Embedded layer - [dimention of vocabulary: 50], [dimention of output vector: 8] [dimention of input sequeances: 4]
* Flatten - reduce to 1 dimention tensor
* Dense - create full connected layer with one ouput neuron (we expect only 1 or 0 as output)

In [33]:
model = Sequential()
model.add(Embedding(vocab_size, 8))
model.build((None, max_length))
model.add(Flatten())
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])
logger.info(model.summary())

Model: "sequential_7"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_7 (Embedding)         │ (None, 4, 8)           │           400 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_6 (Flatten)             │ (None, 32)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_6 (Dense)                 │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 433 (1.69 KB)

 Trainable params: 433 (1.69 KB)

 Non-trainable params: 0 (0.00 B)

2025-04-04 13:18:27.741 | INFO     | __main__:<module>:7 - None


In [34]:
model.fit(padded_docs, labels, epochs=150, verbose=0)
loss, accuracy = model.evaluate(padded_docs, labels, verbose=0)
logger.info('Accuracy: %f' % (accuracy*100))

2025-04-04 13:18:56.525 | INFO     | __main__:<module>:3 - Accuracy: 100.000000
